# User features

## Import packages

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import random
random.seed(32)
import gensim
from gensim import corpora, models
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

## Import data

In [6]:
# authors = pd.read_csv('/home/sophia/ma_py/author_profiles.csv')

df = pd.read_pickle("wordlists_lin_feat_df_withoutuserfeat_allcomments.pkl")
# df = pre_df.set_index(['index', 'author'])
# df.head(10)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1606 entries, -Areopagan- to zyzee
Columns: 103888 entries, body to mrc_pmean
dtypes: float64(87790), int64(16092), object(6)
memory usage: 1.2+ GB


In [4]:
df.tail()

,body,doc_body,probody,tokens,senttokens,agreeableness,openness,conscientiousness,extraversion,neuroticism,...,CNC,IMG,FAM,SER,valence,dominance,negations,articles,mrc_cmean,mrc_pmean
author,,,,,,,,,,,,,,,,,,,,,
zugzwang_03,You know that giggly group of women going to t...,[You know that giggly group of women going to ...,[know giggly group women going bar go dance sa...,"[[know, giggly, group, women, going, bar, go, ...",[[You know that giggly group of women going to...,10.0,41.0,86.0,83.0,18.0,...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN
zuluthrone,"I saw some speculate that the ""download"" would...","[I saw some speculate that the ""download"" woul...",[saw speculate download would backup rather st...,"[[saw, speculate, download, would, backup, rat...","[[I saw some speculate that the ""download"" wou...",17.0,96.0,28.0,95.0,34.0,...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN
zwelg,I am actually pretty pleased about my score:Ag...,[I am actually pretty pleased about my score:A...,[actually pretty pleased score agreeableness m...,"[[actually, pretty, pleased, score, agreeablen...",[[I am actually pretty pleased about my score:...,39.0,89.0,91.0,80.0,3.0,...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN
zymmaster,Respectfully disagree. Offense had plenty of i...,[Respectfully disagree. Offense had plenty of ...,[respectfully disagree offense plenty issues s...,"[[respectfully, disagree, offense, plenty, iss...","[[Respectfully disagree., Offense had plenty o...",28.0,47.0,62.0,21.0,49.0,...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN
zyzee,Tarzaned can't be compared in this situation. ...,[Tarzaned can't be compared in this situation....,[tarzaned not compared situation not pro playe...,"[[tarzaned, not, compared, situation, not, pro...",[[Tarzaned can't be compared in this situation...,88.0,78.0,31.0,75.0,10.0,...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN


## LDA

In [ ]:
# User features

# Preprocessing for LDA
def preprocess_lda(df):
    neglst = ["no", "not", "none", "nobody", "nothing", "neither", "nowhere", "never", "nay"]
    inputlst = []
    for row in tqdm(df['text', 'tokens']):
        rowlst = []
        for comment in row:
            rowlst.append([word for word in comment if (word not in neglst)])
        inputlst.append(rowlst)
    return inputlst
# LDA for topics
def apply_lda(inputlst, number, name):
    print("Start LDA...")
    lst = []
    i = 0
    for row in tqdm(inputlst):
        if len(row) < 2:
            lst.append(-1)
            print("\t-1 appended...")
        else:
            dictionary = corpora.Dictionary(row)
            corpus = [dictionary.doc2bow(text) for text in row]
            print("\tBuild model for row number: ", i)
            ldamodel = gensim.models.LdaMulticore(corpus, num_topics=number, id2word = dictionary, passes=20, workers=16)
            result = ldamodel.print_topics(num_topics=1, num_words=1)
            res = list(result)
            topic = [item[0] for item in res]
            lst.append(topic[0])
            i+=1
#     df[name] = lst
    return lst


print("Preprocessing for LDA...")
inputlst = preprocess_lda(df)
del df
print("LDA with fifty topics: ")
lda50_lst = apply_lda(inputlst, 50, "ldafifty")

In [7]:
df['ldafifty'] = lda50_lst
df.to_pickle("df_withoutlda100.pkl")
# lda50_df.to_pickle("all_feat_withoutlda100_df_allcomments.pkl")
# print("LDA with onehundred topics: ")
# df = apply_lda(lda50_df, inputlst, 100, "ldahundred")
# lda100_df.to_pickle("all_feat_df_allcomments.pkl")


# create df with user features
# user_feat_df = extract_userfeatures(predf)
# user_feat_df.to_pickle("user_feat_df_allcomments.pkl")

In [8]:
del df
del lda50_lst

# Wrapper

In [ ]:
print("Load new df...")
newdf = pd.read_pickle("df_withoutlda100.pkl")
newdf.info()



def preprocess_lda(df):
    neglst = ["no", "not", "none", "nobody", "nothing", "neither", "nowhere", "never", "nay"]
    inputlst = []
    for row in tqdm(df['tokens']):
        rowlst = []
        for comment in row:
            rowlst.append([word for word in comment if (word not in neglst)])
        inputlst.append(rowlst)
    return inputlst


def apply_lda(df, inputlst, number, name):
    print("Start LDA...")
    lst = []
    for row in tqdm(inputlst):
        if len(row) < 2:
            lst.append(-1)
            print("\t-1 appended...")
        else:
            print("\tBuild dictionary for row...")
            dictionary = corpora.Dictionary(row)
            print("\tBuild corpus for row...")
            corpus = [dictionary.doc2bow(text) for text in row]
            print("\tBuild model for row...")
            ldamodel = gensim.models.LdaMulticore(corpus, num_topics=number, id2word = dictionary, passes=20, workers=10)
            print("\tCalculate result for row...")
            result = ldamodel.print_topics(num_topics=1, num_words=1)
            res = list(result)
            print("\tDelete unnecessary information...")
            topic = [item[0] for item in res]
            lst.append(topic[0])
            print("\tDone with this row...")
    df[name] = lst
    return df

print("Preprocessing for LDA...")
newinputlst = preprocess_lda(newdf)


print("LDA with onehundred topics: ")
df = apply_lda(newdf, newinputlst, 100, "ldahundred")

In [3]:
df.to_pickle("allfeat_df_allcomments.pkl")

In [10]:
del newdf

In [5]:
df.info(verbose=True)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

